In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import os
import joblib
import streamlit

In [3]:
# Ruta del archivo CSV
file_path =r'/Users/Camilo/Documents/tfm/TFM_inmobiliario_main/TFM_inmobiliario/datamunging/consolidated_data.csv' 
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(file_path, sep = ',')
# Verificar el tamaño del DataFrame
print(f"Tamaño del DataFrame: {df.shape}")
# Eliminar filas donde 'precio' es NaN, 0, o infinito
df = df[~df['precio'].isin([0, np.inf, -np.inf]) & df['precio'].notna()]
# Separar características y variable objetivo
X = df.drop('precio', axis=1)
y = df['precio']
# Imputar valores NaN en y con la mediana
y = y.fillna(y.median())
# Identificar columnas numéricas
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
# Preprocesamiento para columnas numéricas
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])
# Combinar preprocesadores solo para columnas numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols)
    ])
# Crear pipelines para RandomForest y LinearRegression
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X[num_cols], y, test_size=0.2, random_state=42)
# Verificar tamaños de conjuntos de entrenamiento y prueba
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")
# Entrenar y evaluar el modelo de RandomForest
rf_pipeline.fit(X_train, y_train)

joblib.dump(rf_pipeline, 'modelo_test.pkl')
y_pred_rf = rf_pipeline.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

/var/folders/6l/xmxqxb352dl57_n06gxvgs3m0000gn/T/ipykernel_41801/1176395177.py:4: DtypeWarning: Columns (11,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep = ',')


Tamaño del DataFrame: (284168, 18)
Tamaño de X_train: (225525, 7)
Tamaño de X_test: (56382, 7)


In [4]:
X.columns

Index(['sub_descr', 'href', 'ubicacion', 'habitaciones', 'banios', 'mt2',
       'otros', 'latitude', 'longitude', 'planta', 'CCAA', 'fuente_datos',
       'alquiler_venta', 'fecha_extract', 'publicado_hace', 'plataforma',
       'raw_json'],
      dtype='object')